In [1]:
# !python -m wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [14]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [15]:
print('length of dataset in characters:', len(text))

length of dataset in characters: 1115394


In [16]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [17]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(repr(''.join(chars)))
print(vocab_size)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
65


In [18]:
stoi = {c: i for i, c in enumerate(chars)}
itos = {i: c for i, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('string to encode'))
print(decode(encode('string to encode')))

[57, 58, 56, 47, 52, 45, 1, 58, 53, 1, 43, 52, 41, 53, 42, 43]
string to encode


In [19]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [20]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [21]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [22]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'for context {context} target is {target}')

for context tensor([18]) target is 47
for context tensor([18, 47]) target is 56
for context tensor([18, 47, 56]) target is 57
for context tensor([18, 47, 56, 57]) target is 58
for context tensor([18, 47, 56, 57, 58]) target is 1
for context tensor([18, 47, 56, 57, 58,  1]) target is 15
for context tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
for context tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [24]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx (B,T)
        # targets (B,T)

        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx (B,T)

        for _ in range(max_new_tokens):
            logits, _ = self(idx) # (B,T,C)
            logits = logits[:, -1, :] # (B,C)
            probs = F.softmax(logits, dim=-1) # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)

        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(logits)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor([[ 0.4224, -0.8596,  0.2910,  ...,  0.2380, -0.1270,  0.0534],
        [ 0.8564,  2.2181,  0.5232,  ...,  0.4114,  1.9312,  1.0119],
        [ 0.4851,  0.0060,  1.0007,  ..., -0.0776, -0.5015, -2.2270],
        ...,
        [-0.1726, -0.6626, -0.5495,  ..., -0.3630,  0.4550,  0.7595],
        [ 1.0923, -1.3024,  1.3521,  ..., -0.2658, -1.4075, -0.7460],
        [ 0.4851,  0.0060,  1.0007,  ..., -0.0776, -0.5015, -2.2270]],
       grad_fn=<ViewBackward0>)
tensor(4.8865, grad_fn=<NllLossBackward0>)

uoiaF$z
M?kI;h
DbuMG,H3LYNmrDxKgTpvAKOF-jU.hc;fBMTGa-IS
g3lEb&ZQ,l;:m;lpcNN
KpVEYRIIM,'hCRbMAcWTkrnH


In [28]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [34]:
batch_size = 32
for _ in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.480619430541992


In [35]:
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


An:
RCAMy lowind anil te buthy wal yothelire I d, bes l plofee touly RCHeatoven parar yond montoue,

